prepare data for CNN
* use the extracted corrdinate and area, using plt to resize the data

In [57]:
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import os
import re

In [58]:
input_dir = "../coordinate_radius"
sub_dir = "../resize_image"
output_dir = "../"
# unify the image size
image_size = (10, 10)
pixels = []
targets = []

# use loop to read the extracted coordinates and radius 
for class_name in os.listdir(input_dir):
    if not class_name.endswith('.png'):
        continue
    class_path = os.path.join(input_dir, class_name)
    star_df = pd.read_csv(class_path)
    star_df = star_df.drop(columns=["Unnamed: 0"])
    max_value = np.max(star_df.iloc[:, :2].max())
    rounded_max_values = np.ceil(max_value / 50) * 50 # rounded_max_value for axis range
    # ursa_minor_1 --> ursa_minor
    pattern = r"^(.*?)_\d+\.png$"
    match = re.match(pattern, class_name) 
    if match:
        target = match.group(1)
    else:
        target = class_name
    targets.append(target)  

    # draw the unifying picture
    sky_image = plt.figure(figsize=image_size, dpi=100)
    ax = sky_image.add_subplot(111)
    for index, row in star_df.iterrows():
        x, y, radius = row['x'], row['y'], row['r']
        circle = plt.Circle((x, y), radius, color='white', alpha=1.0, edgecolor='none', zorder=10)
        ax.add_artist(circle)
    ax.set_xlim(0, rounded_max_values)
    ax.set_ylim(0, rounded_max_values)
    ax.set_aspect('equal')
    ax.set_facecolor('black') 
    ax.set_xticks([])
    ax.set_yticks([])
    subpath = os.path.join(sub_dir, class_name)
    plt.savefig(subpath, bbox_inches='tight', pad_inches=0, dpi=100)

    # extract the pixel information for unified image
    image = Image.open(subpath)
    image_array = np.array(image)
    pixels.append(image_array)

# store the pixel and target in numpy (just like mnist datset)
pixels = np.array(pixels)
targets = np.array(targets)
np.save(output_dir + "images.npy", pixels)
np.save(output_dir + "labels.npy", targets)